<a href="https://colab.research.google.com/github/spatiebalk/text_mining_project/blob/master/text_generation_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation

In this notebook text will be generated with multiple different methods, as well as multiple different kinds of text (poetry, news artciles, stories). The generated text will be evaluated using the BERTscore.

In [ ]:
import numpy as np
import tensorflow as tf
import os
from os.path import join, isfile
import time
import csv
from tqdm import tqdm
import json
from os import listdir
from os.path import isfile, join
import sys

In [ ]:
import sys
print(sys.version)
print(tf.__version__)

3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
2.4.0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Harry Potter data

In [ ]:
text = open('/content/gdrive/My Drive/TxMM/harrypotter.txt').read()

### 1. GRU|
From https://medium.com/towards-artificial-intelligence/create-your-own-harry-potter-short-story-using-rnn-and-tensorflow-853b3ed1b8f3 and https://www.tensorflow.org/tutorials/text/text_generation

In [ ]:
# print(text[:200])

In [ ]:
vocab = sorted(set(text))
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)
text_as_int = np.array([char2index[c] for c in text])

In [ ]:
seq_length = 50
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(data):
  input_text = data[:-1]
  target_text = data[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 300

# Number of RNN units 
rnn_units1 = 512
rnn_units2 = 256
rnn_units= [rnn_units1, rnn_units2]

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
       batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
vocab_size = vocab_size,
embedding_dim=embedding_dim,
rnn_units=rnn_units,
batch_size=BATCH_SIZE)

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,
         logits, from_logits=True)
  
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 300)           31800     
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1250304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 256)           591360    
_________________________________________________________________
dense (Dense)                (64, None, 106)           27242     
Total params: 1,900,706
Trainable params: 1,900,706
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/gdrive/My Drive/TxMM/GRU_results/training_checkpoints_GRU_hp'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_prefix, save_weights_only=True)

In [ ]:
EPOCHS= 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
latest_check = tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/50
1915/1915 [==============================] - 44s 21ms/step - loss: 1.9830 - accuracy: 0.4482
Epoch 2/50
1915/1915 [==============================] - 41s 21ms/step - loss: 1.3330 - accuracy: 0.6033
Epoch 3/50
1915/1915 [==============================] - 42s 22ms/step - loss: 1.2774 - accuracy: 0.6175
Epoch 4/50
1915/1915 [==============================] - 43s 22ms/step - loss: 1.2519 - accuracy: 0.6240
Epoch 5/50
1915/1915 [==============================] - 43s 22ms/step - loss: 1.2370 - accuracy: 0.6281
Epoch 6/50
1915/1915 [==============================] - 44s 23ms/step - loss: 1.2255 - accuracy: 0.6308
Epoch 7/50
1915/1915 [==============================] - 44s 23ms/step - loss: 1.2181 - accuracy: 0.6330
Epoch 8/50
1915/1915 [==============================] - 44s 22ms/step - loss: 1.2131 - accuracy: 0.6342
Epoch 9/50
1915/1915 [==============================] - 44s 22ms/step - loss: 1.2088 - accuracy: 0.6355
Epoch 10/50
1915/1915 [==============================] - 44s 23m

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(latest_check)
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 300)            31800     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 512)            1250304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 256)            591360    
_________________________________________________________________
dense_1 (Dense)              (1, None, 106)            27242     
Total params: 1,900,706
Trainable params: 1,900,706
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):

  num_generate = 1000  #can be anything you like
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  scaling = 0.5 #kept at a lower value here
  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / scaling
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # Pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(index2char[predicted_id])

  return (start_string + "".join(text_generated))

In [ ]:
print(generate_text(model, start_string= "Severus Snape"))

Severus Snape was about the bank. And they spectain to the wand and her from them behind him again on the side shock to a good or his face formite that the should face the first of the Mudble soundle and the end the gravents was sister from the parch of the scape be close that he could not thought the sounding and the strang and the dark for them and she had the past that the sward of Ron was start and a long study and surprised and shaker of the sittly fall and the travels and more was thing of the stood sharish, and they are the door on the suspected him.
"Who into the close to see the thing that an experious of Magic that the said and fell when the parting and Grey could not the she was not seem to Griphoop. And he was in the pocter on the same back over his face of the thought he moved to stude the stop in the stair of the potion. He was a destronumition of the stood the blumbled and a laugher finer and some of she saw the secide somether to call of the spart of the bitter of his f

In [ ]:
# open prompts file
prompts = open('/content/gdrive/My Drive/TxMM/hp_prompts.txt', 'r').readlines()
generated_dict = {}

for i, p in tqdm(enumerate(prompts)):
  # generate 1000 characters
  gen_text = generate_text(model, start_string=p)
  generated_dict[i] = gen_text

with open('/content/gdrive/My Drive/TxMM/GRU_results/hp_generated_texts.json', 'w') as fp:
  json.dump(generated_dict, fp)

100it [10:57,  6.58s/it]


### News data

In [ ]:
news_dir = '/content/gdrive/My Drive/TxMM/news_data'
files = [f for f in listdir(news_dir) if isfile(join(news_dir, f))]
text = ""

csv.field_size_limit(sys.maxsize)

for f in files:
  with open(join(news_dir, f)) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in tqdm(csv_reader):
      if line_count == 0:
        print(f'Column names are {", ".join(row)}')
        line_count += 1
      else:
        text = text + " " + row[-1] + " "
        line_count += 1
  break

with open('/content/gdrive/My Drive/TxMM/news_data.txt', 'w') as outfile:
    
    outfile.write(text)

132it [00:00,  5.27it/s]

Column names are , id, title, publication, author, date, year, month, url, content


50001it [3:20:35,  4.15it/s]


In [ ]:
vocab = sorted(set(text))
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)
text_as_int = np.array([char2index[c] for c in text])

In [ ]:
seq_length = 50
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(data):
  input_text = data[:-1]
  target_text = data[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 300

# Number of RNN units 
rnn_units1 = 512
rnn_units2 = 256
rnn_units= [rnn_units1, rnn_units2]

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
       batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
vocab_size = vocab_size,
embedding_dim=embedding_dim,
rnn_units=rnn_units,
batch_size=BATCH_SIZE)

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,
         logits, from_logits=True)
  
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 300)           300600    
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1250304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 256)           591360    
_________________________________________________________________
dense (Dense)                (64, None, 1002)          257514    
Total params: 2,399,778
Trainable params: 2,399,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/gdrive/My Drive/TxMM/GRU_results/training_checkpoints_GRU_news'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_prefix, save_weights_only=True)

In [ ]:
EPOCHS= 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
latest_check = tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/50
59060/59060 [==============================] - 1600s 27ms/step - loss: 1.7411 - accuracy: 0.5072
Epoch 2/50
59060/59060 [==============================] - 1597s 27ms/step - loss: 1.4119 - accuracy: 0.5890
Epoch 3/50
59060/59060 [==============================] - 1596s 27ms/step - loss: 1.3943 - accuracy: 0.5932
Epoch 4/50
59060/59060 [==============================] - 1599s 27ms/step - loss: 1.3932 - accuracy: 0.5932
Epoch 5/50
59060/59060 [==============================] - 1595s 27ms/step - loss: 1.4035 - accuracy: 0.5900
Epoch 6/50
59060/59060 [==============================] - 1596s 27ms/step - loss: 1.6018 - accuracy: 0.5336
Epoch 7/50
59060/59060 [==============================] - 1631s 28ms/step - loss: 1.6774 - accuracy: 0.5140
Epoch 8/50
59060/59060 [==============================] - 1618s 27ms/step - loss: 1.5197 - accuracy: 0.5584
Epoch 9/50
59060/59060 [==============================] - 1623s 27ms/step - loss: 1.4943 - accuracy: 0.5653
Epoch 10/50
59060/59060 [===

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(latest_check)
model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
# open prompts file
prompts = open('/content/gdrive/My Drive/TxMM/news_prompts.txt', 'r').readlines()
generated_dict = {}

for i, p in tqdm(enumerate(prompts)):
  # generate 1000 characters
  gen_text = generate_text(model, start_string=p)
  generated_dict[i] = gen_text

with open('/content/gdrive/My Drive/TxMM/GRU_results/hp_generated_texts.json', 'w') as fp:
  json.dump(generated_dict, fp)